# How to Use:
This Notebook should ease the use of the API for "cleanup" tasks. It includes some sample for specific use cases that can be adjusted.
1. run first block to prepare excecutions by creating api connection
2. run any following block to execute a specific function

In [ ]:
import logging
import logging.config
import json
from pathlib import Path
from churchtools_api.churchtools_api import ChurchToolsApi

logger = logging.getLogger(__name__)

config_file = Path("../logging_config.json")
with config_file.open(encoding="utf-8") as f_in:
    logging_config = json.load(f_in)
    log_directory = Path(logging_config["handlers"]["file"]["filename"]).parent
    if not log_directory.exists():
        log_directory.mkdir(parents=True)
    logging.config.dictConfig(config=logging_config)

# Create Session
from secure.config import ct_domain
from secure.config import ct_token
api = ChurchToolsApi(ct_domain)
api.login_ct_rest_api(ct_token=ct_token)

## Songs & Tag - mass change
Helper to iterate all songs

In [ ]:
songs = api.get_songs()
print('Got {} songs and the first one is "{}"'.format(len(songs),songs[0]['name']))

Mass change append a tag to all songs - e.g. 51 is tag:"in ChurchTools vor Skript Import"

In [ ]:
all_song_ids = [value['id'] for value in songs]
for id in all_song_ids:
    api.add_song_tag(id, 51)